## 作業
礙於不是所有同學都有 GPU ，這邊的範例使用的是簡化版本的 ResNet，確保所有同學都能夠順利訓練!


最後一天的作業請閱讀這篇非常詳盡的[文章](https://blog.gtwang.org/programming/keras-resnet-50-pre-trained-model-build-dogs-cats-image-classification-system/)，基本上已經涵蓋了所有訓練　CNN 常用的技巧，請使用所有學過的訓練技巧，盡可能地提高 Cifar-10 的 test data 準確率，截圖你最佳的結果並上傳來完成最後一次的作業吧!

另外這些技巧在 Kaggle 上也會被許多人使用，更有人會開發一些新的技巧，例如使把預訓練在 ImageNet 上的模型當成 feature extractor 後，再拿擷取出的特徵重新訓練新的模型，這些技巧再進階的課程我們會在提到，有興趣的同學也可以[參考](https://www.kaggle.com/insaff/img-feature-extraction-with-pretrained-resnet)

In [2]:
import keras
from keras.layers import Dense, Conv2D, BatchNormalization, Activation,MaxPool2D,Dropout
from keras.layers import AveragePooling2D, Input, Flatten
from keras.optimizers import Adam,SGD
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras import backend as K
from keras.models import Model
from keras.datasets import cifar10
import numpy as np
import os

os.environ['CUDA_VISIBLE_DEVICES']='0'
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 5541723717127263720
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 6702159318943243398
physical_device_desc: "device: XLA_CPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 4817616896
locality {
  bus_id: 1
  links {
  }
}
incarnation: 1205657121032570305
physical_device_desc: "device: 0, name: GeForce RTX 2060, pci bus id: 0000:01:00.0, compute capability: 7.5"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 2621356366263822657
physical_device_desc: "device: XLA_GPU device"
]


In [4]:
# 讀取 Cifar-10 資料集
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# 影像輸入的維度
input_shape = x_train.shape[1:]

# 先把影像縮放到 0-1 之間
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

# 再減去所有影像的平均值
#if subtract_pixel_mean:
#    x_train_mean = np.mean(x_train, axis=0)
#   x_train -= x_train_mean 
#    x_test -= x_train_mean # 此處要注意！測試資料也是減去訓練資料的平均值來做標準化，不可以減測試資料的平均值 (因為理論上你是不能知道測試資料的平均值的！)

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
print('y_train shape:', y_train.shape)


EPOCHS = 100
BATCHSIZE = 25
LR = 1e-3
class_unit = 10 


# 對 label 做 one-hot encoding
y_train = keras.utils.to_categorical(y_train, class_unit)
y_test = keras.utils.to_categorical(y_test, class_unit)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples
y_train shape: (50000, 1)


In [5]:
model = keras.Sequential()
model.add(Conv2D(32,(3,3),padding='same',input_shape = x_train.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size = (2,2)))
model.add(Dropout(0.3))

model.add(Conv2D(64,(3,3),padding='same'))
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size = (2,2)))
model.add(Dropout(0.3))

model.add(Conv2D(64,(3,3),padding='same'))
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size = (2,2)))
model.add(Dropout(0.3))

model.add(Conv2D(128,(3,3),padding='same'))
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size = (2,2)))
model.add(Dropout(0.3))

model.add(Flatten())
model.add(Dense(1024))
model.add(Activation('relu'))
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(class_unit))
model.add(Activation('softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 32)        896       
_________________________________________________________________
activation (Activation)      (None, 32, 32, 32)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 16, 64)        18496     
_________________________________________________________________
activation_1 (Activation)    (None, 16, 16, 64)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 8, 8, 64)          0

In [6]:
model.compile(loss= 'categorical_crossentropy',
            optimizer=Adam(),
            metrics=['accuracy'])

In [8]:
history = model.fit(x_train,y_train,
            batch_size=BATCHSIZE,
            epochs=EPOCHS,
            verbose= 1,
            validation_data=(x_test,y_test)
            )

Epoch 1/100
2000/2000 [==============================] - 6s 3ms/step - loss: 1.5183 - accuracy: 0.4438 - val_loss: 1.3268 - val_accuracy: 0.5177
Epoch 2/100
2000/2000 [==============================] - 6s 3ms/step - loss: 1.3892 - accuracy: 0.4982 - val_loss: 1.3161 - val_accuracy: 0.5247
Epoch 3/100
2000/2000 [==============================] - 6s 3ms/step - loss: 1.3291 - accuracy: 0.5231 - val_loss: 1.1884 - val_accuracy: 0.5763
Epoch 4/100
2000/2000 [==============================] - 6s 3ms/step - loss: 1.2722 - accuracy: 0.5435 - val_loss: 1.2328 - val_accuracy: 0.5479
Epoch 5/100
2000/2000 [==============================] - 6s 3ms/step - loss: 1.2303 - accuracy: 0.5609 - val_loss: 1.1349 - val_accuracy: 0.5971
Epoch 6/100
2000/2000 [==============================] - 6s 3ms/step - loss: 1.2018 - accuracy: 0.5717 - val_loss: 1.0995 - val_accuracy: 0.6065
Epoch 7/100
2000/2000 [==============================] - 6s 3ms/step - loss: 1.1752 - accuracy: 0.5854 - val_loss: 1.1292 - val_ac

In [11]:
score = model.evaluate(x_test,y_test,verbose = 0)
print('loss:',score[0],
        '\naccuracy:',score[1])

loss: 0.7608503103256226 
accuracy: 0.75


In [12]:
model.save('./cifar10_1101_v1')

INFO:tensorflow:Assets written to: ./cifar10_1101_v1\assets
